## Ao longo da execucao, documente o codigo com comentarios, referentes ao seu entendimento. Utilize tambem a estrutura de markdown do Jupyter para colocar informacoes textuais relevantes.

In [ ]:
from __future__ import print_function

import sys
if sys.version >= '3':
    long = int

from pyspark.sql import SparkSession

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
spark = SparkSession\
        .builder\
        .appName("ALSExample")\
        .config("spark.mongodb.read.connection.uri", "mongodb://172.20.0.2:27017/puc.recomendacoes") \
        .config("spark.mongodb.write.connection.uri", "mongodb://172.20.0.2:27017/puc.recomendacoes") \
        .config('spark.jars.packages',"org.mongodb.spark:mongo-spark-connector_2.12:10.3.0")\
        .getOrCreate()

In [ ]:
lines = spark.read.text("sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=long(p[3])))
ratings = spark.createDataFrame(ratingsRDD.collect())

In [ ]:
ratings.show()

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [ ]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
              coldStartStrategy="drop")
model = als.fit(training)

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

In [ ]:
userRecs = model.recommendForAllUsers(10)

In [ ]:
userRecs.show(10, False)

In [ ]:
movieRecs = model.recommendForAllItems(10)

In [ ]:
movieRecs.show()

In [ ]:
users = ratings.select(als.getUserCol()).distinct()

In [ ]:
users.show()

In [ ]:
userRecsOnlyItemId = userRecs.select(userRecs['userId'], userRecs['recommendations']['movieId'])

In [ ]:
userRecsOnlyItemId.show(10, False)

In [ ]:
userRecs.select(userRecs["userId"], \
                userRecs["recommendations"]["movieId"].alias("movieId"),\
userRecs["recommendations"]["rating"].cast('array<double>').alias("rating")).\
    write.format("mongodb").mode("append").save()

## Agora faça 50 recomendacoes para todos os usuarios

## Recomende 50 usuarios para os itens

## Como poderiamos armazenar as recomendacoes no MongoDB?

## Como podemos fazer isso em Python?

## Podemos utilizar outros datasets de teste, em especifico do proprio MovieLenz? Pesquise sobre esses datasets e sua estrutura basica.

## Amazene os dados no MongoDB, criem consultas para obter as recomendações para os usuários de ID 6 e 20.